# DeePTB Tutorial 1: deeptb-sk baseline model [v2.2]

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DeePTB-Lab/Recipes/blob/main/deeptb_tutorials/v2.2/DeePTB_Tutorial_1_Colab.ipynb)

<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:1rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        Author: <a style="font-weight:normal" href="mailto:guqq@ustc.edu.cn">Gu, Qiangqiang 顾强强 📨 </a></b></i><br/>
        Date: 2025-04-20<br/>
        Protocol：<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License.</a></i><br/>
        Quick Start：Click the <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;">Connect</span> button，then run all cells.
    </p>
</div>

## 🚀 Installation (Colab/Binder)

This cell will automatically detect your environment and install DeePTB using UV package manager.

In [ ]:
# ============================================
# DeePTB 自动安装脚本 (使用UV)
# ============================================
import sys
import os
from pathlib import Path

# 检测运行环境
IN_COLAB = 'google.colab' in sys.modules
IN_BINDER = 'BINDER_SERVICE_HOST' in os.environ
IN_LOCAL = not (IN_COLAB or IN_BINDER)

print("="*60)
print("🔍 环境检测")
print("="*60)
if IN_COLAB:
    print("✅ 检测到 Google Colab 环境")
elif IN_BINDER:
    print("✅ 检测到 Binder 环境")
else:
    print("✅ 检测到本地环境")

# 检查是否已安装DeePTB
try:
    import dptb
    print(f"\n✅ DeePTB 已安装 (版本: {dptb.__version__})")
    DEEPTB_INSTALLED = True
except ImportError:
    print("\n⚠️  DeePTB 未安装,开始安装流程...")
    DEEPTB_INSTALLED = False

# 在线环境需要安装
if (IN_COLAB or IN_BINDER) and not DEEPTB_INSTALLED:
    print("\n" + "="*60)
    print("📦 开始安装 DeePTB")
    print("="*60)
    
    # 步骤1: 安装UV
    print("\n[1/4] 安装 UV 包管理器...")
    !pip install -q uv
    print("✅ UV 安装完成")
    
    # 步骤2: 克隆DeePTB仓库
    print("\n[2/4] 克隆 DeePTB 仓库...")
    if not Path('DeePTB').exists():
        !git clone -q https://github.com/deepmodeling/DeePTB.git
        print("✅ 仓库克隆完成")
    else:
        print("✅ DeePTB 仓库已存在")
    
    # 步骤3: 使用UV安装DeePTB
    print("\n[3/4] 使用 UV 安装 DeePTB 及依赖...")
    print("⏳ 这可能需要几分钟,请耐心等待...")
    
    # 检测CUDA版本
    if IN_COLAB:
        # Colab通常有CUDA 12.1
        cuda_version = "cu121"
        print(f"🎮 检测到GPU环境,使用 CUDA {cuda_version}")
    else:
        # Binder通常是CPU
        cuda_version = "cpu"
        print("💻 使用 CPU 版本")
    
    # 切换到DeePTB目录并安装
    %cd DeePTB
    !uv sync --find-links https://data.pyg.org/whl/torch-2.5.0+{cuda_version}.html
    print("✅ DeePTB 安装完成")
    
    # 步骤4: 验证安装
    print("\n[4/4] 验证安装...")
    !uv run dptb --version
    
    # 返回原目录
    %cd ..
    
    print("\n" + "="*60)
    print("🎉 安装完成!")
    print("="*60)
    
elif IN_LOCAL and not DEEPTB_INSTALLED:
    print("\n" + "="*60)
    print("⚠️  本地环境检测")
    print("="*60)
    print("请在本地环境中手动安装 DeePTB:")
    print("")
    print("  git clone https://github.com/deepmodeling/DeePTB.git")
    print("  cd DeePTB")
    print("  uv sync")
    print("")
    print("详细安装说明: https://github.com/deepmodeling/DeePTB#installation")
    print("="*60)

## 📁 Download Tutorial Data

Download the required data files for this tutorial.

In [ ]:
# 下载教程数据
import os
from pathlib import Path

if IN_COLAB or IN_BINDER:
    print("📥 下载教程数据文件...")
    
    # 克隆Recipes仓库获取数据
    if not Path('Recipes').exists():
        !git clone -q https://github.com/DeePTB-Lab/Recipes.git
        print("✅ 数据文件下载完成")
    
    # 切换到tutorial目录
    %cd Recipes/deeptb_tutorials/v2.2
    print(f"📂 当前工作目录: {os.getcwd()}")
else:
    print("💻 本地环境,使用现有数据文件")
    print(f"📂 当前工作目录: {os.getcwd()}")

## Introduction

**DeePTB** is a method that uses deep learning to accelerate first-principles electronic structure simulations.

### Version Features
- **v1**: Constructed tight-binding (TB) models with first-principles accuracy (DeePTB-SK)
- **v2**: Added E3 equivariant networks to represent single-electron operators (Hamiltonian, density matrix, and overlap matrix) (DeePTB-E3)
- **v2.2**: Incorporated built-in SK empirical parameters covering commonly used elements across the periodic table

Through these capabilities, DeePTB provides multiple approaches to accelerate electronic structure simulations of materials.

### Learning Objectives

In this tutorial, you will:
1. Learn how to use built-in base model to plot band structure for given crystal structure
2. Learn how to generate a empirical sk model in deeptb-sk format for target system

## 1. Calculating Band Structure for a Given Structure

The deeptb-sk module now [since v2.2] has built-in empirical SK parameter models covering elements across the periodic table. 

These can be directly used to obtain empirical SKTB models for given structures. It also supports directly plotting band structures for a given structure.

In [ ]:
# 检查数据目录
!ls ./data/

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

current_dir = os.getcwd()
workdir = f'{current_dir}/data/base_model/'
os.chdir(f"{workdir}/structures")

# 使用ls代替tree (Colab可能没有tree命令)
!ls -la ./